 # Agente LangChain - Agente de Análise de Cliente para Marketing Telco

 Este notebook implementa um agente que analisa informações de clientes para gerar ações de marketing.

Este agente utiliza sinais gerados por modelos de IA tradicional (churn e valor mensal) para gerar insights de negócio para Marketing.

Ele não toma decisões automáticas.
Ele explica contextos e sugere caminhos.

 ## Coletor de notícias do seu time

In [ ]:
!pip install langchain-oci langchain



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
from langchain_core.tools import tool
from typing import Dict, List
import re
from tavily import TavilyClient

@tool
def get_time_comprehensive_news(pergunta: str) -> str:
    """Realiza buscas na web em múltiplas fontes do GE Globo."""
    
    client = TavilyClient("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") ## Pode usar esse mesmo, tem limite de 100 requests para o workshop
    response = client.search(
    query=pergunta)
    return response



## Adicionando tool de data atual

In [ ]:
@tool
def get_today_date() -> str:
    """Retorna a data de hoje no formato DD/MM/AAAA"""
    from datetime import datetime
    return datetime.now().strftime("%d/%m/%Y")

 ## Configuração de autenticação de OCI SDK
    - https://github.com/oracle/oci-python-sdk/blob/master/examples/configuration_example.py

 ## Criação do Agente LangChain

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_oci.chat_models import ChatOCIGenAI
from langchain.agents import create_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import AIMessage
import json



class CleanAgentCallback(BaseCallbackHandler):
    def on_tool_start(self, serialized, input_str, **kwargs):
        tool_name = serialized.get("name", "unknown_tool")
        print(f"\n TOOL: {tool_name}")
        print(f"   ↳ input: {input_str}")

    def on_tool_end(self, output, **kwargs):
        print("TOOL RESULT (resumo):")

        # output pode vir como ToolMessage ou string
        content = getattr(output, "content", output)

        # tenta resumir JSON grande
        try:
            data = json.loads(content)
            results = data.get("results", [])
            print(f"   ↳ {len(results)} fontes encontradas")

            if results:
                top = results[0]
                print(f"   ↳ destaque: {top.get('title')}")
        except Exception:
            # fallback simples
            text = str(content)
            print("   ↳", text[:200], "..." if len(text) > 200 else "")



# 2. Configuração da LLM OCI
llm = ChatOCIGenAI(
  model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyajqi26fkxly6qje5ysvezzrypapl7ujdnqfjq6hzo2loq",
  service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
  compartment_id="ocid1.compartment.oc1..aaaaaaaa3x7n7wwfnghe4imvt3niwo76wgqv6ecn2iadiwoph73jjowbhbna",
  provider="meta",
  model_kwargs={
    "temperature": 0.3, 
    "max_tokens": 800,   
    "top_p": 0.8,  
    "frequency_penalty": 0,
    "presence_penalty": 0,
  },
  auth_type="API_KEY",
  auth_profile="CHICAGOV2"
)
tools = [
    get_time_comprehensive_news,
    get_today_date
]

react_prompt_template = """You are a helpful assistant specialized in Brazilian football teams news and information. 
Answer the following questions as best you can using the available tools.

IMPORTANT: After using any tool, you MUST analyze and summarize the results in your own words. 
Do not just copy the raw output - provide a meaningful summary and insights. Do not use old news.

You have access to the following tools:

{tools}

Use the following format EXACTLY:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of the tools
Action Input: the input to the action (use empty string "" if no input needed)
Observation: the result of the action
Thought: Now I need to analyze this information and provide a helpful summary
Final Answer: A comprehensive answer based on the tool results, with analysis and insights in Brazilian Portuguese

Begin!

"""


agent = create_agent(llm, tools, system_prompt=react_prompt_template)


pergunta = f"Qual próximo jogo do {time}?"

callback = CleanAgentCallback()

def extract_final_output(agent_response):
    final_text = None
    total_tokens = None

    for msg in reversed(agent_response["messages"]):
        if isinstance(msg, AIMessage):
            if msg.content and final_text is None:
                if "Final Answer:" in msg.content:
                    final_text = msg.content.split("Final Answer:", 1)[1].strip()
                else:
                    final_text = msg.content.strip()

            if total_tokens is None:
                total_tokens = (
                    msg.response_metadata.get("total_tokens")
                    or msg.additional_kwargs.get("total_tokens")
                )

        if final_text and total_tokens:
            break

    return final_text, total_tokens

response = agent.invoke(
    {"messages": pergunta},
    config={"callbacks": [callback]}
)

final_answer, total_tokens = extract_final_output(response)

print("\nFinal answer:")
print(final_answer)

print("\nMetadados:")
print(f"total tokens: {total_tokens}")




 TOOL: get_time_comprehensive_news
   ↳ input: {'pergunta': 'próximo jogo do Flamengo'}
TOOL RESULT (resumo):
   ↳ 5 fontes encontradas
   ↳ destaque: Quais os próximos jogos do Flamengo? Veja datas e horários

Final answer:
O próximo jogo do Flamengo está marcado para o dia 17 de Dezembro, às 14h (horário de Brasília), contra o Paris Saint-Germain (PSG) na final da Copa Intercontinental 2025. Além disso, o time também tem jogos agendados para os dias 21, 24 e 31 de Janeiro, e 4 e 7 de Fevereiro, contra times como Bangu-RJ, Volta Redonda, Vasco da Gama, Fluminense, Portuguesa-RJ e Sampaio Corrêa-RJ. É importante verificar os horários e locais dos jogos, pois podem estar sujeitos a mudanças.

Metadados:
total tokens: 1733


In [ ]:
pergunta = f"De quanto foi o ultimo jogo {time}"
response = agent.invoke(
    {"messages": pergunta},
    config={"callbacks": [callback]}
)

final_answer, total_tokens = extract_final_output(response)

print("\nFinal answer:")
print(final_answer)

print("\nMetadados:")
print(f"total tokens: {total_tokens}")


 TOOL: get_time_comprehensive_news
   ↳ input: {'pergunta': 'De quanto foi o ultimo jogo Flamengo'}
TOOL RESULT (resumo):
   ↳ 5 fontes encontradas
   ↳ destaque: Flamengo - Resultados - ESPN (BR)

Final answer:
O último jogo do Flamengo foi contra o Cruz Azul, e o Flamengo venceu por 2 a 1, avançando para a semifinal da Copa Intercontinental.

Metadados:
total tokens: 2078
